# Часть 1

1) Извлечем zip архив

In [31]:
import zipfile

def extract_encrypted_zip(zip_file_path, password, destination_path):
    try:
        with zipfile.ZipFile(zip_file_path) as zip_file:
            zip_file.setpassword(bytes(password, 'utf-8'))
            zip_file.extractall(path=destination_path)
            print("Файлы успешно извлечены")
    except:
        print(f"Error: Произошла ошибка во время извлечения файла")

In [32]:
zip_file_path = 'protected_archive.zip'
file_name = 'invoice-42369643.html'
destination_path = 'extracted_files/'
password = 'netology'
extract_encrypted_zip(zip_file_path, password, destination_path)

Файлы успешно извлечены


2) Отправим файл в VirusTotal

In [3]:
api_key = input()

In [4]:
import requests

def send_to_virus_total(file_path):
    api_url = "https://www.virustotal.com/api/v3/files"
    headers = {"x-apikey" : api_key}
    with open(file_path, "rb") as file:
        files = {"file": (file_path, file)}
        response = requests.post(api_url, headers=headers, files=files)
        return response.json()

In [5]:
file_path = 'extracted_files/invoice-42369643.html'
vt_response = send_to_virus_total(file_path)
analysis_id = vt_response['data']['id']

3) Посмотрим на детальный анализ данного файла

In [33]:
import time
def get_analysis_info(analysis_id):
    headers = {"x-apikey" : api_key}
    url_analysis_details = f'https://www.virustotal.com/api/v3/analyses/{analysis_id}'
    response_analysis = requests.get(url_analysis_details, headers=headers)
    time.sleep(30)
    if response_analysis.status_code == 200:
        analysis_details = response_analysis.json()
        return analysis_details
    else:
        print('Ошибка при получении деталей анализа:', response_analysis.text)
        return ""
analysis_info = get_analysis_info(analysis_id)

4) Выведем список угроз, которые мы обнаружили

In [34]:
print("Detected:")
for values in analysis_info['data']['attributes']['results'].values():
    if 'category' in values and values['category'] == 'malicious':
        print(values['engine_name'], values['result'])

Detected:
Lionic Trojan.Script.Obfuse.4!c
MicroWorld-eScan Trojan.Generic.37575014
CTX txt.trojan.obfuse
ALYac Trojan.Generic.37575014
Symantec Trojan.Gen.MBT
ESET-NOD32 VBA/TrojanDropper.Agent.AXB
Kaspersky UDS:DangerousObject.Multi.Generic
Arcabit Trojan.Generic.D23D5966
Emsisoft Trojan.Generic.37575014 (B)
DrWeb X97M.DownLoader.459
VIPRE Trojan.Generic.37575014
FireEye Trojan.Generic.37575014
Kingsoft Win32.Infected.AutoInfector.a
Microsoft Trojan:O97M/Obfuse.CP
ViRobot HTML.Z.Agent.48950
GData Trojan.Generic.37575014
Google Detected
Ikarus Trojan-Dropper.VBA.Agent
Fortinet VBA/Agent.BCF0!tr
alibabacloud Trojan[dropper]:MSOffice/Obfuse.CX


# Часть 2

1) Десериализуем данный json

In [35]:
import json

json_data = '''
[
    {
        "Program": "LibreOffice",
        "Version": "6.0.7"
    },
    {
        "Program": "7zip",
        "Version": "18.05"
    },
    {
        "Program": "Adobe Reader",
        "Version": "2018.011.20035"
    },
    {
        "Program": "nginx",
        "Version": "1.14.0"
    },
    {
        "Program": "Apache HTTP Server",
        "Version": "2.4.29"
    },
    {
        "Program": "DjVu Reader",
        "Version": "2.0.0.27"
    },
    {
        "Program": "Wireshark",
        "Version": "2.6.1"
    },
    {
        "Program": "Notepad++",
        "Version": "7.5.6"
    },
    {
        "Program": "Google Chrome",
        "Version": "68.0.3440.106"
    },
    {
        "Program": "Mozilla Firefox",
        "Version": "61.0.1"
    }
]
'''

data = json.loads(json_data)

In [36]:
vulners_api_key = input()

2) Получим список уязвимостей ПО, используя API Vulners

In [37]:
import vulners

vulners_api = vulners.Vulners(api_key=vulners_api_key)

program_version = []
for element in data:
    program_version.append({
        "product": element['Program'],
        "version": element['Version']
    })

response = vulners_api.audit_software(
    software=program_version,
    fields=["title", "short_description"],
    match='partial'
)

response

/home/user1/anaconda3/lib/python3.12/site-packages/vulners/vulners.py:838: DeprecationWarning: Vulners is deprecated and will be removed in future release. Use VulnersApi instead.
  warnings.warn(


[{'input': {'product': 'Adobe Reader', 'version': '2018.011.20035'},
  'matched_criteria': 'cpe:2.3:a:adobe:reader:2018.011.20035:*:*:*:*:*:*:*',
  'vulnerabilities': []},
 {'input': {'product': 'LibreOffice', 'version': '6.0.7'},
  'matched_criteria': 'cpe:2.3:a:libreoffice:libreoffice:6.0.7:*:*:*:*:*:*:*',
  'vulnerabilities': [{'id': 'CVE-2019-9848',
    'reasons': [{'config': 'nvd',
      'criterias': [[{'criteria': 'cpe:2.3:a:libreoffice:libreoffice:*:*:*:*:*:*:*:*',
         'vulnerable': True,
         'versionEndExcluding': '6.2.5'}]]}],
    'title': 'CVE-2019-9848',
    'short_description': 'CVE-2019-9848: LibreOffice prior to 6.2.5 allows execution of arbitrary python commands through document events'},
   {'id': 'CVE-2019-9849',
    'reasons': [{'config': 'nvd',
      'criterias': [[{'criteria': 'cpe:2.3:a:libreoffice:libreoffice:*:*:*:*:*:*:*:*',
         'vulnerable': True,
         'versionEndExcluding': '6.2.5'}]]}],
    'title': 'CVE-2019-9849',
    'short_description':

3) Получим список ПО-уязвимости

In [38]:
for item in response:
    print(f"Name: {item['input']['product']}")
    for vuln in item['vulnerabilities']:
        print(vuln['id'])

Name: Adobe Reader
Name: LibreOffice
CVE-2019-9848
CVE-2019-9849
CVE-2019-9850
CVE-2019-9851
CVE-2019-9852
CVE-2020-12802
CVE-2020-12803
CVE-2019-9847
Name: Wireshark
CVE-2018-14339
CVE-2018-14340
CVE-2018-14341
CVE-2018-14342
CVE-2018-14343
CVE-2018-14344
CVE-2018-14367
CVE-2018-14368
CVE-2018-14369
CVE-2018-14370
CVE-2019-10894
CVE-2019-10895
CVE-2019-10896
CVE-2019-10899
CVE-2019-10901
CVE-2019-10903
CVE-2019-19553
CVE-2018-14438
CVE-2018-18225
CVE-2018-18226
CVE-2018-18227
CVE-2020-13164
CVE-2020-11647
CVE-2020-25862
CVE-2020-25863
CNVD-2024-39270
CVE-2019-5716
CVE-2019-5717
CVE-2019-5718
CVE-2019-5719
CVE-2018-16056
CVE-2018-16057
CVE-2018-16058
CNVD-2021-11320
CVE-2019-9208
CVE-2019-9209
CVE-2019-9214
CVE-2019-13619
CVE-2019-16319
CVE-2020-9428
CVE-2020-9430
CVE-2020-9431
CVE-2019-12295
CVE-2020-26575
CVE-2018-19622
CVE-2018-19623
CVE-2018-19624
CVE-2018-19625
CVE-2018-19626
CVE-2018-19627
CVE-2018-19628
CVE-2023-2906
Name: Apache HTTP Server
CVE-2018-1301
CVE-2018-1302
CVE-2018-